In [1]:
import nltk, string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import pickle
import os
import re
import pandas as pd

In [2]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]


'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    text1 = removeSpecialChars(text1)
    text2 = removeSpecialChars(text2)
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

def removeSpecialChars(text):
    return re.sub("[^a-zA-Z0-9]", " ", text)

In [3]:
text = 'Converting a Pandas GroupBy object to DataFrame Python Pandas - Re-ordering columns in a dataframe'

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [5]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [6]:
def process_text(text):
    # Tokenize
    tokens = word_tokenize(text)
    # remove all tokens that are not alphabetic. Special Characters
    words = [word for word in tokens if word.isalpha()]
    # Remove stopwords
    words = [w for w in words if not w in stop_words]
    # stemming of words
    stemmed = [porter.stem(word) for word in words]
    processed_text = (' ').join(stemmed)
    return processed_text

In [7]:
from collections import deque

# Class to extract method as well as attribute calls. Each token after '.' is called attribute 
# be it function call or anything else

class AttributeVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_all_calls(tree):
    all_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Attribute):
            callvisitor = AttributeVisitor()
            callvisitor.visit(node)
            all_calls.append(callvisitor.name)
    return all_calls

# Visitin method calls only
class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

In [8]:
so_dump_processed_file = '../data/stack-overflow/pandas-preprocessedcode-dataset-part3'
dataset ='../data/stack-overflow/Dataset - Pandas.csv'
api_doc_file = '../code/data-import/build_api_doc_base/api_doc.csv'
id_col = 'Id' 

In [9]:
processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)
api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)
dataset_df = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

In [10]:
api_df.to_csv('api_df.csv', index=False)

In [11]:
## Get API description with fully qualified name for a method from API doc and build the context
def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['Description']
            tokens = row['FullyQualifiedName'].split('.')
        
            for token in tokens:
                methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print(e)
    return api_dict
        
## Get AnswerId and Question Text combo from dataset to build the context
def buildAnswerIdQuestionTextDict(dataset_df):
    dataset_answerId_QText_Dict = dict()
    try:
        for idx, row in dataset_df.iterrows():
            answerId = row['AnswerId']
            if answerId != 0:
                dataset_answerId_QText_Dict[answerId] = process_text(row['QuestionText'])
    except Exception as e:
        print(e)
        
    return dataset_answerId_QText_Dict

api_dict = buildAPIDictionary(api_df)
dataset_answerId_QText_Dict = buildAnswerIdQuestionTextDict(dataset_df)

In [12]:
def lookUpAPIDocForContext(method_name):
    if method_name in api_dict.keys():
        return api_dict[method_name]
    else:
        return ""

def getSOContext(answerId):
    return dataset_answerId_QText_Dict[int(answerId)]

In [13]:
processed_stackoverflow_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'AcceptedAnswerId', 'AnswerCount', 'Body',
       'ClosedDate', 'CommentCount', 'CommunityOwnedDate', 'CreationDate',
       'FavoriteCount', 'Id', 'LastActivityDate', 'LastEditDate',
       'LastEditorDisplayName', 'LastEditorUserId', 'OwnerDisplayName',
       'OwnerUserId', 'ParentId', 'PostTypeId', 'Score', 'Tags', 'Title',
       'ViewCount', 'Code', 'PreprocessedCode', 'PreprocessedCode2',
       'PreprocessedCode2_1', 'PreprocessedCode3'],
      dtype='object')

In [14]:
title_codeElement_pair_df = pd.DataFrame(columns=['title', 'all_calls'])
for index, row in processed_stackoverflow_df.iterrows():
    try:
        if row.PostTypeId !=1:
            tile = getSOContext(row.Id)
            code_snippet = row.PreprocessedCode3
            tree = ast.parse(code_snippet)
            all_calls = get_all_calls(tree)
            if(len(all_calls) !=0):
                title_codeElement_pair_df.loc[index] = [tile, all_calls]
            
    except KeyError:
        pass

In [15]:
title_codeElement_pair_df.head()

,title,all_calls
1209,panda datafram find row valu column maxim,"[pandas.DataFrame, df.argmax, df.argmax, df.ar..."
1211,convert panda groupbi object datafram,"[g1.index, g1.add_suffix.reset_index, df1.grou..."
1213,redefin index panda datafram object,[data.set_index]
1232,python panda column datafram base column name,"[df.reindex_axis, df.columns]"
1238,databas like mysql,"[cx_Oracle.connect, pd.read_sql, ora_conn.clos..."


In [16]:
def filter_code_elements_using_H1(df):
    all_calls = df['all_calls']
    filtered_calls = []
    for call in all_calls:
        tokens = call.split('.')
        method_name = tokens[len(tokens)-1]
        if lookUpAPIDocForContext(method_name):
            filtered_calls.append(call)
    df['h1_filtered_calls'] = filtered_calls
    return df

In [17]:
title_codeElement_pair_df_after_h1 = title_codeElement_pair_df.apply(filter_code_elements_using_H1, axis=1)

In [18]:
title_codeElement_pair_df_after_h1

,title,all_calls,h1_filtered_calls
1209,panda datafram find row valu column maxim,"[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[pandas.DataFrame, df.argmax, df.argmax, df.ar..."
1211,convert panda groupbi object datafram,"[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou..."
1213,redefin index panda datafram object,[data.set_index],[data.set_index]
1232,python panda column datafram base column name,"[df.reindex_axis, df.columns]",[df.reindex_axis]
1238,databas like mysql,"[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]"
...,...,...,...
2856,drop duplic row python panda,"[pd.DataFrame, df.drop_duplicates]","[pd.DataFrame, df.drop_duplicates]"
2859,remov row duplic indic panda datafram timeseri,"[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]"
2951,remov index column panda,[df.read_csv],[df.read_csv]
3060,how split column two column,"[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, pd.DataFrame, df.str.split, pd...."


In [19]:
def buildAPIDictionaryForH2(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['SubCategory']
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict

api_dict_H2 = buildAPIDictionaryForH2(api_df)

In [20]:
def lookUpAPIDocForContextH2(method_name):
    try:
        if method_name in api_dict_H2.keys() and api_dict_H2[method_name] == "Constructor":
            return True
        else:
            return False
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)

In [21]:
def filter_code_elements_using_H2(df):
    all_calls = df['all_calls']
    filtered_calls = []
    for call in all_calls:
        tokens = call.split('.')
        method_name = tokens[len(tokens)-1]
        if not lookUpAPIDocForContextH2(method_name):
            filtered_calls.append(call)
    df['h2_filtered_calls'] = filtered_calls
    return df

In [22]:
title_codeElement_pair_df_after_h2 = title_codeElement_pair_df_after_h1.apply(filter_code_elements_using_H2, axis=1)

In [23]:
title_codeElement_pair_df_after_h2

,title,all_calls,h1_filtered_calls,h2_filtered_calls
1209,panda datafram find row valu column maxim,"[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[df.argmax, df.argmax, df.argmax, dfrm.idxmax,..."
1211,convert panda groupbi object datafram,"[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou..."
1213,redefin index panda datafram object,[data.set_index],[data.set_index],[data.set_index]
1232,python panda column datafram base column name,"[df.reindex_axis, df.columns]",[df.reindex_axis],"[df.reindex_axis, df.columns]"
1238,databas like mysql,"[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]","[cx_Oracle.connect, pd.read_sql, ora_conn.clos..."
...,...,...,...,...
2856,drop duplic row python panda,"[pd.DataFrame, df.drop_duplicates]","[pd.DataFrame, df.drop_duplicates]",[df.drop_duplicates]
2859,remov row duplic indic panda datafram timeseri,"[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]"
2951,remov index column panda,[df.read_csv],[df.read_csv],[df.read_csv]
3060,how split column two column,"[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, pd.DataFrame, df.str.split, pd....","[df.str.split.str, df.str.split.str, df.str.sp..."


In [24]:
def filter_code_elements_using_H1H2(df):
    # H1 filtered call
    h1_filtered_calls = df['h1_filtered_calls']
    len_h1 = len(h1_filtered_calls)
    filtered_calls = []
    for call in h1_filtered_calls:
        tokens = call.split('.')
        method_name = tokens[len(tokens)-1]
        if not lookUpAPIDocForContextH2(method_name):
            filtered_calls.append(call)
    df['h1h2_filtered_calls'] = filtered_calls
    len_h1h2 = len(filtered_calls)
    return df

In [25]:
title_codeElement_pair_df_after_h1h2 = title_codeElement_pair_df_after_h2.apply(filter_code_elements_using_H1H2, axis=1)

In [26]:
title_codeElement_pair_df_after_h1h2

,title,all_calls,h1_filtered_calls,h2_filtered_calls,h1h2_filtered_calls
1209,panda datafram find row valu column maxim,"[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[df.argmax, df.argmax, df.argmax, dfrm.idxmax,...","[df.argmax, df.argmax, df.argmax, dfrm.idxmax,..."
1211,convert panda groupbi object datafram,"[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou..."
1213,redefin index panda datafram object,[data.set_index],[data.set_index],[data.set_index],[data.set_index]
1232,python panda column datafram base column name,"[df.reindex_axis, df.columns]",[df.reindex_axis],"[df.reindex_axis, df.columns]",[df.reindex_axis]
1238,databas like mysql,"[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]","[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]"
...,...,...,...,...,...
2856,drop duplic row python panda,"[pd.DataFrame, df.drop_duplicates]","[pd.DataFrame, df.drop_duplicates]",[df.drop_duplicates],[df.drop_duplicates]
2859,remov row duplic indic panda datafram timeseri,"[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]"
2951,remov index column panda,[df.read_csv],[df.read_csv],[df.read_csv],[df.read_csv]
3060,how split column two column,"[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, pd.DataFrame, df.str.split, pd....","[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, df.str.split, upper_lower_df.st..."


In [27]:
def filter_code_elements_using_M1(df, cosine_sim_threshould):
    all_calls = df['all_calls']
    len_all = len(all_calls)
    filtered_calls = []
    for call in all_calls:
        tokens = call.split('.')
        method_name = tokens[len(tokens)-1]
        api_context = lookUpAPIDocForContext(method_name)
        if api_context == "":
            cos_score = -1
        else:
            cos_score = cosine_sim(api_context, df['title'])
        if cos_score > cosine_sim_threshould:
            filtered_calls.append(call)
    df['m1_filtered_calls'] = filtered_calls
    return df

In [28]:
title_codeElement_pair_df_after_m1 = title_codeElement_pair_df_after_h1h2.apply(filter_code_elements_using_M1, args=(0.1,), axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yo

In [29]:
title_codeElement_pair_df_after_m1

,title,all_calls,h1_filtered_calls,h2_filtered_calls,h1h2_filtered_calls,m1_filtered_calls
1209,panda datafram find row valu column maxim,"[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[pandas.DataFrame, df.argmax, df.argmax, df.ar...","[df.argmax, df.argmax, df.argmax, dfrm.idxmax,...","[df.argmax, df.argmax, df.argmax, dfrm.idxmax,...","[pandas.DataFrame, dfrm.idxmax, dfrm.idxmax]"
1211,convert panda groupbi object datafram,"[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.index, g1.add_suffix.reset_index, df1.grou...","[g1.add_suffix.reset_index, df1.groupby.size.D..."
1213,redefin index panda datafram object,[data.set_index],[data.set_index],[data.set_index],[data.set_index],[data.set_index]
1232,python panda column datafram base column name,"[df.reindex_axis, df.columns]",[df.reindex_axis],"[df.reindex_axis, df.columns]",[df.reindex_axis],[]
1238,databas like mysql,"[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]","[cx_Oracle.connect, pd.read_sql, ora_conn.clos...","[pd.read_sql, pd.read_sql]",[]
...,...,...,...,...,...,...
2856,drop duplic row python panda,"[pd.DataFrame, df.drop_duplicates]","[pd.DataFrame, df.drop_duplicates]",[df.drop_duplicates],[df.drop_duplicates],"[pd.DataFrame, df.drop_duplicates]"
2859,remov row duplic indic panda datafram timeseri,"[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]","[df3.index.duplicated, df3.index]",[df3.index.duplicated]
2951,remov index column panda,[df.read_csv],[df.read_csv],[df.read_csv],[df.read_csv],[]
3060,how split column two column,"[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, pd.DataFrame, df.str.split, pd....","[df.str.split.str, df.str.split.str, df.str.sp...","[df.str.split, df.str.split, upper_lower_df.st...","[df.str.split, pd.DataFrame, df.str.split, pd...."


In [30]:
def list_to_text(df, columns):
    for col in columns:
        list_ = df[col]
        df[col] = " ".join(list_)
    return df

In [31]:
list_columns = ['all_calls', 'h1_filtered_calls', 'h2_filtered_calls', 'h1h2_filtered_calls', 'm1_filtered_calls']
title_codeElement_pair = title_codeElement_pair_df_after_m1.apply(list_to_text, args=(list_columns,), axis=1)

In [32]:
title_codeElement_pair

,title,all_calls,h1_filtered_calls,h2_filtered_calls,h1h2_filtered_calls,m1_filtered_calls
1209,panda datafram find row valu column maxim,pandas.DataFrame df.argmax df.argmax df.argmax...,pandas.DataFrame df.argmax df.argmax df.argmax...,df.argmax df.argmax df.argmax dfrm.idxmax dfrm...,df.argmax df.argmax df.argmax dfrm.idxmax dfrm...,pandas.DataFrame dfrm.idxmax dfrm.idxmax
1211,convert panda groupbi object datafram,g1.index g1.add_suffix.reset_index df1.groupby...,g1.index g1.add_suffix.reset_index df1.groupby...,g1.index g1.add_suffix.reset_index df1.groupby...,g1.index g1.add_suffix.reset_index df1.groupby...,g1.add_suffix.reset_index df1.groupby.size.Dat...
1213,redefin index panda datafram object,data.set_index,data.set_index,data.set_index,data.set_index,data.set_index
1232,python panda column datafram base column name,df.reindex_axis df.columns,df.reindex_axis,df.reindex_axis df.columns,df.reindex_axis,
1238,databas like mysql,cx_Oracle.connect pd.read_sql ora_conn.close M...,pd.read_sql pd.read_sql,cx_Oracle.connect pd.read_sql ora_conn.close M...,pd.read_sql pd.read_sql,
...,...,...,...,...,...,...
2856,drop duplic row python panda,pd.DataFrame df.drop_duplicates,pd.DataFrame df.drop_duplicates,df.drop_duplicates,df.drop_duplicates,pd.DataFrame df.drop_duplicates
2859,remov row duplic indic panda datafram timeseri,df3.index.duplicated df3.index,df3.index.duplicated df3.index,df3.index.duplicated df3.index,df3.index.duplicated df3.index,df3.index.duplicated
2951,remov index column panda,df.read_csv,df.read_csv,df.read_csv,df.read_csv,
3060,how split column two column,df.str.split.str df.str.split.str df.str.split...,df.str.split pd.DataFrame df.str.split pd.Data...,df.str.split.str df.str.split.str df.str.split...,df.str.split df.str.split upper_lower_df.str.l...,df.str.split pd.DataFrame df.str.split pd.Data...


In [33]:
title_codeElement_pair.columns

Index(['title', 'all_calls', 'h1_filtered_calls', 'h2_filtered_calls',
       'h1h2_filtered_calls', 'm1_filtered_calls'],
      dtype='object')

In [35]:
title_codeElement_pair.to_csv('corpus/title.txt', columns = ['title'], index=False, header=False)
title_codeElement_pair.to_csv('corpus/all_calls_code.txt', columns = ['all_calls'], index=False, header=False)
title_codeElement_pair.to_csv('corpus/h1_filtered_calls_code.txt', columns = ['h1_filtered_calls'], index=False, header=False)
title_codeElement_pair.to_csv('corpus/h2_filtered_calls_code.txt', columns = ['h2_filtered_calls'], index=False, header=False)
title_codeElement_pair.to_csv('corpus/h1h2_filtered_calls_code.txt', columns = ['h1h2_filtered_calls'], index=False, header=False)
title_codeElement_pair.to_csv('corpus/m1_filtered_calls_code.txt', columns = ['m1_filtered_calls'], index=False, header=False)